# salesForecasting
- Sara Echeverría 21371
- Ricardo Mendez 21289
- Melissa Pérez 21385

Repository link: https://github.com/bl33h/salesForecasting

## Task 1

## Task 2

### 1. ¿Cuál es el problema del gradiente de fuga en las redes LSTM y cómo afecta la efectividad de LSTM para el pronóstico de series temporales?
En este contexto, al gradiente utilizado en las LSTM dificulta aprender dependencias a largo plazo. Su arquitectura ayudn a retener información relevante, mejorando el pronóstico de series temporales.

### 2. ¿Cómo se aborda la estacionalidad en los datos de series temporales cuando se utilizan LSTM para realizar pronósticos y qué papel juega la diferenciación en el proceso?
Esta se maneja por medio de la diferenciación para eliminar patrones estacionales, permitiendo un pronóstico más efectivo al enfocar las dependencias temporales significativas.

### 3. ¿Cuál es el concepto de "tamaño de ventana" en el pronóstico de series temporales con LSTM y cómo afecta la elección del tamaño de ventana a la capacidad del modelo para capturar patrones a corto y largo plazo?
Este hace referencia a la cantidad de datos pasados que usa el LSTM para predecir futuros valores. Ventanas más grandes capturan patrones a largo plazo, mientras que ventanas más pequeñas se enfocan en patrones recientes. En ambos casos puede haber pérdida de información.